In [2]:
import PIL
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
import Augmentor
import os
import cv2
from pascal_voc_writer import Writer
import shutil
import numpy as np

import random
from random import randrange

import Augmentor


import numpy
def find_coeffs(source_coords, target_coords):
    matrix = []
    for s, t in zip(source_coords, target_coords):
        matrix.append([t[0], t[1], 1, 0, 0, 0, -s[0]*t[0], -s[0]*t[1]])
        matrix.append([0, 0, 0, t[0], t[1], 1, -s[1]*t[0], -s[1]*t[1]])
    A = numpy.matrix(matrix, dtype=numpy.float)
    B = numpy.array(source_coords).reshape(8)
    res = numpy.dot(numpy.linalg.inv(A.T * A) * A.T, B)
    return numpy.array(res).reshape(8)

In [135]:
#Create Number, Letters and Year-Numbers

os.makedirs('./chars/letters/',exist_ok=True)
os.makedirs('./chars/numbers_small/',exist_ok=True)
os.makedirs('./chars/numbers_big/',exist_ok=True)

lets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
num = '0123456789'


for x in lets:
    font = ImageFont.truetype("stuff/Sauerkrauto W00 Regular.ttf",58)
    img=Image.open('template_char.jpg')
    draw = ImageDraw.Draw(img)
    draw.text((4, -11),x,(0,0,0),font=font)
    draw = ImageDraw.Draw(img)
    img.save('chars/letters/'+str(x)+".jpg")
    
for x in num:
    font = ImageFont.truetype("stuff/Sauerkrauto W00 Regular.ttf",58)
    img=Image.open('template_char.jpg')
    draw = ImageDraw.Draw(img)
    draw.text((4, -11),x,(0,0,0),font=font)
    draw = ImageDraw.Draw(img)
    img.save('./chars/numbers_big/'+str(x)+".jpg")
    

for x in num:
    l1 = random.choice(num)
    font_num = ImageFont.truetype("stuff/Uniform Extra Condensed.ttf",35)
    img=Image.open('template_char_small.jpg')
    draw = ImageDraw.Draw(img)
    draw.text((1, -1),x,(0,0,0),font=font_num)
    draw = ImageDraw.Draw(img)
    img.save('./chars/numbers_small/'+str(x)+".jpg")

In [136]:
p = Augmentor.Pipeline("./chars/letters/")
p.skew_tilt(1,0.2)
p.skew_corner(1,0.2)
p.random_distortion(0.8,2,2,1)
p.shear(0.7,10,10)
p.rotate(0.8,5,10)
p.sample(26000)

p = Augmentor.Pipeline("./chars/numbers_big/")
p.skew_tilt(1,0.1)
p.skew_corner(1,0.1)
p.random_distortion(0.8,2,2,1)
p.shear(0.7,5,5)
p.rotate(0.8,5,10)
p.sample(10000)

p = Augmentor.Pipeline("./chars/numbers_small/")
p.skew_tilt(1,0.1)
p.skew_corner(1,0.1)
p.random_distortion(0.8,2,2,1)
p.shear(0.7,5,5)
p.rotate(0.8,5,10)
p.sample(10000)

Executing Pipeline:   0%|                                                              | 0/26000 [00:00<?, ? Samples/s]

Initialised with 26 image(s) found.
Output directory set to ./chars/letters/output.

Processing <PIL.Image.Image image mode=RGB size=40x55 at 0x207AF2393C8>: 100%|█| 26000/26000 [01:01<00:00, 426.04 Sampl
Executing Pipeline:   0%|                                                              | 0/10000 [00:00<?, ? Samples/s]

Initialised with 10 image(s) found.
Output directory set to ./chars/numbers_big/output.

Processing <PIL.Image.Image image mode=RGB size=40x55 at 0x207AE0E5688>: 100%|█| 10000/10000 [00:29<00:00, 342.89 Sampl
Executing Pipeline:   0%|                                                              | 0/10000 [00:00<?, ? Samples/s]

Initialised with 10 image(s) found.
Output directory set to ./chars/numbers_small/output.

Processing <PIL.Image.Image image mode=RGB size=15x30 at 0x207B04ACCC8>: 100%|█| 10000/10000 [00:26<00:00, 372.26 Sampl


In [137]:
import glob
import shutil
import os

src_dir = "./chars/numbers_small/output"
dst_dir = "./chars/numbers_small/"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)
shutil.rmtree(src_dir)

src_dir = "./chars/numbers_big/output"
dst_dir = "./chars/numbers_big/"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)
shutil.rmtree(src_dir)

src_dir = "./chars/letters/output"
dst_dir = "./chars/letters/"
for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)
shutil.rmtree(src_dir)

In [208]:
lbl_path = './train_char/lbls/'
imgs_pth = './train_char/imgs/'
os.makedirs(lbl_path,exist_ok=True)
os.makedirs(imgs_pth,exist_ok=True)

for x in range(1000):
    writer = Writer(lbl_path+str(x)+'.jpg', 480,480)
    img = Image.open('./chars/letters/'+random.choice(os.listdir('./chars/letters/'))).convert("RGBA")
    bg_plate=Image.open('template_blank.png')
    bg_plate.paste(img, (70,8), img)
    writer.addObject('Char', int(72),int(10),int(72+35),int(10+50))
    
    img = Image.open('./chars/letters/'+random.choice(os.listdir('./chars/letters/'))).convert("RGBA")
    bg_plate.paste(img, (120,8), img)
    writer.addObject('Char', int(122),int(10),int(122+35),int(10+50))
    
    img = Image.open('./chars/letters/'+random.choice(os.listdir('./chars/letters/'))).convert("RGBA")
    bg_plate.paste(img, (170,8), img)
    writer.addObject('Char', int(172),int(10),int(172+35),int(10+50))
    
    img = Image.open('./chars/numbers_big/'+random.choice(os.listdir('./chars/numbers_big/'))).convert("RGBA")
    bg_plate.paste(img, (70,70), img)
    writer.addObject('Char', int(72),int(72),int(72+35),int(72+50))
    
    img = Image.open('./chars/numbers_big/'+random.choice(os.listdir('./chars/numbers_big/'))).convert("RGBA")
    bg_plate.paste(img, (120,70), img)
    writer.addObject('Char', int(122),int(72),int(122+35),int(72+50))
    
    img = Image.open('./chars/numbers_big/'+random.choice(os.listdir('./chars/numbers_big/'))).convert("RGBA")
    bg_plate.paste(img, (170,70), img)
    writer.addObject('Char', int(172),int(72),int(172+35),int(72+50))
    
    img = Image.open('./chars/numbers_small/'+random.choice(os.listdir('./chars/numbers_small/'))).convert("RGBA")
    bg_plate.paste(img, (220,50), img)
    writer.addObject('Char', int(220),int(50),int(220+15),int(50+30))
    
    img = Image.open('./chars/numbers_small/'+random.choice(os.listdir('./chars/numbers_small/'))).convert("RGBA")
    bg_plate.paste(img, (240,50), img)
    writer.addObject('Char', int(240),int(50),int(240+15),int(50+30))
    
    
    bg_rand = Image.open('./bgs/'+random.choice(os.listdir('./bgs/'))).convert("RGBA")
    bg_rand = bg_rand.resize((480,480), Image.ANTIALIAS)
    bg_rand.paste(bg_plate.convert("RGBA"), (100,200), bg_plate.convert("RGBA"))
    
    opencvImage = cv2.cvtColor(numpy.array(bg_rand), cv2.COLOR_RGB2BGR)
    w,h,_ = opencvImage.shape
#     cv2.rectangle(opencvImage,(int(220),int(50)),(int(220+15),int(50+30)),(0,255,0),1)
#     cv2.rectangle(opencvImage,(int(240),int(50)),(int(240+15),int(50+30)),(0,255,0),1)
#     cv2.rectangle(opencvImage,(int(122),int(72)),(int(122+35),int(72+50)),(0,255,0),1)
#     cv2.rectangle(opencvImage,(int(122),int(10)),(int(122+35),int(10+50)),(0,255,0),1)
    
    cv2.imwrite(imgs_pth+str(x)+".jpg",opencvImage)
    writer.save(lbl_path+str(x)+'.xml')
    
    
import os
import xml.etree.ElementTree as ET
import pandas as pd
df = pd.DataFrame(columns=['img','xmin','ymin','w','h','class'])


for x in os.listdir(lbl_path):
    if x.endswith('.xml'):
        tree = ET.parse(lbl_path+x)
        root = tree.getroot()
        img = root[1].text[:-4]
        
        country= tree.findall('.//object')
        for y in range(len(country)):
            xmin = country[y][4][0].text
            xmin = int(xmin)+100
            
            ymin = country[y][4][1].text
            ymin = int(ymin)+200
            
            w = country[y][4][2].text
            w= int(w)+100
            
            h = country[y][4][3].text
            h = int(h)+200
            
            df.loc[len(df)] = [img,xmin,ymin,w,h,0]
            
df.to_csv('./train_char/train.csv',index=False)

In [209]:
df.head()

,img,xmin,ymin,w,h,class
0,0,172,210,207,260,0
1,0,222,210,257,260,0
2,0,272,210,307,260,0
3,0,172,272,207,322,0
4,0,222,272,257,322,0


### To separate out the Classes - Only run to make the Character Recognition Model Data

In [138]:
all_lets = os.getcwd()+'/chars/letters/'
for x in os.listdir(all_lets):
    for l in lets:
        if '_'+str(l)+'.jpg' in x:
            os.makedirs('./chars_recog/'+str(l),exist_ok=True)
            shutil.move(all_lets+x,'./chars_recog/'+str(l))

In [139]:
all_nums = os.getcwd()+'/chars/numbers_big/'
for n in os.listdir(all_nums):
    for m in num:
        if '_'+str(m)+'.jpg' in n:
            os.makedirs('./chars_recog/'+str(m),exist_ok=True)
            shutil.move(all_nums+n,'./chars_recog/'+str(m))
            break

In [140]:
all_yrs = './chars/numbers_small/'
for n in os.listdir(all_yrs):
    for m in num:
        if '_'+str(m)+'.jpg' in n and len(n) > 5:
            os.makedirs('./chars_recog/'+str(m),exist_ok=True)
            shutil.move(all_yrs+n,'./chars_recog/'+str(m))
            break

In [154]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
 
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
 
    # return the edged image
    return edged
img = cv2.imread('./yolov5/yolov5/crp.png')
print(img.shape)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
thresh_inv = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV,39,1)
edges = auto_canny(thresh_inv)
ctrs, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
img_area = img.shape[0]*img.shape[1]
for i, ctr in enumerate(sorted_ctrs):
    x, y, w, h = cv2.boundingRect(ctr)
    roi_area = w*h
    roi_ratio = roi_area/img_area
# if((roi_ratio >= 0.015) and (roi_ratio < 0.09)):
    print(roi_ratio)
#     if ((h>1.2*w) and (3*w>=h)):
#         print('rect')
    cv2.rectangle(img,(x,y),( x + w, y + h ),(90,0,255),2)

(143, 256, 3)
0.0021033653846153845
0.0003277972027972028
0.06539554195804195
0.10817307692307693
0.0015570367132867133
0.004097465034965035
0.007211538461538462
0.11888111888111888
0.0013111888111888112
0.004015515734265734
5.463286713286713e-05
0.00010926573426573427
0.0013111888111888112
0.00021853146853146853
0.0002458479020979021
0.0002458479020979021
0.4871612762237762
5.463286713286713e-05
0.0006555944055944056
5.463286713286713e-05
2.7316433566433566e-05
2.7316433566433566e-05
2.7316433566433566e-05
2.7316433566433566e-05
0.001966783216783217
0.00010926573426573427
0.0001638986013986014
0.00010926573426573427
5.463286713286713e-05
2.7316433566433566e-05
8.19493006993007e-05
0.0002458479020979021
5.463286713286713e-05
0.007867132867132868
0.010052447552447552
0.0027316433566433565
0.0003277972027972028
8.19493006993007e-05
5.463286713286713e-05
0.00704763986013986
0.009560751748251748
0.0009833916083916085
2.7316433566433566e-05
0.004097465034965035
5.463286713286713e-05
0.00016

In [251]:
img = cv2.imread('./yolov5/yolov5/crp.png')
(h, w) = img.shape[:2]
image_size = h*w
mser = cv2.MSER_create()
mser.setMaxArea(int(image_size/10))
mser.setMinArea(100)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Converting to GrayScale
_, bw = cv2.threshold(gray, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

regions, rects = mser.detectRegions(bw)

# With the rects you can e.g. crop the letters
for (x, y, w, h) in rects:
    if 1 > (w/h) > 0.2 and (w*h) < 1000:
        1==1
#         cv2.rectangle(img, (x, y), (x+w, y+h), color=(255, 0, 255), thickness=1)

In [252]:
def bounding_rect_to_contour(c):
    (x,y,w,h) = cv2.boundingRect(c)
    return ( (x,y,w,h), np.asarray([[[x,y]],[[x+w,y]],[[x+w,y+h]],[[x,y+h]]]) )

In [253]:
conts=[]
for (x,y,w,h) in rects:
    conts.append(np.asarray([[[x,y]],[[x+w,y]],[[x+w,y+h]],[[x,y+h]]]))
img = cv2.drawContours(img,conts,-1,(0,255,0),1)

In [254]:
cv2.imwrite('mm.png',img)

True